# ECG Model Training

This notebook demonstrates training deep learning models for ECG signal classification.

## Models:
1. 1D Convolutional Neural Network (CNN)
2. LSTM-based model
3. Hybrid CNN-LSTM model

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

import warnings
warnings.filterwarnings('ignore')

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {tf.config.list_physical_devices('GPU')}")

## Load and Prepare Data

In [ ]:
# Generate synthetic data for demonstration
# In practice, this would be loaded from preprocessed ECG segments
np.random.seed(42)

n_samples = 5000
n_timesteps = 360  # 1 second at 360 Hz
n_classes = 5  # Normal, PVC, PAC, VF, VT

# Generate synthetic ECG segments
X = np.random.randn(n_samples, n_timesteps)
y = np.random.randint(0, n_classes, n_samples)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

# Normalize
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Reshape for CNN (add channel dimension)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

# One-hot encode labels
y_train_cat = to_categorical(y_train, n_classes)
y_val_cat = to_categorical(y_val, n_classes)
y_test_cat = to_categorical(y_test, n_classes)

print(f"Training set: {X_train.shape}")
print(f"Validation set: {X_val.shape}")
print(f"Test set: {X_test.shape}")
print(f"Number of classes: {n_classes}")

## Model 1: 1D Convolutional Neural Network

In [ ]:
def build_cnn_model(input_shape, n_classes):
    """Build 1D CNN model for ECG classification"""
    model = models.Sequential([
        # First convolutional block
        layers.Conv1D(64, kernel_size=7, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        # Second convolutional block
        layers.Conv1D(128, kernel_size=5, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        # Third convolutional block
        layers.Conv1D(256, kernel_size=3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        # Dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(n_classes, activation='softmax')
    ])
    
    return model

# Build model
cnn_model = build_cnn_model((n_timesteps, 1), n_classes)
cnn_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(cnn_model.summary())

In [ ]:
# Define callbacks
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7
)

# Train model
history_cnn = cnn_model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nCNN Training completed!")

## Model 2: LSTM Model

In [ ]:
def build_lstm_model(input_shape, n_classes):
    """Build LSTM model for ECG classification"""
    model = models.Sequential([
        layers.LSTM(128, return_sequences=True, input_shape=input_shape),
        layers.Dropout(0.3),
        
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.3),
        
        layers.LSTM(32),
        layers.Dropout(0.3),
        
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(n_classes, activation='softmax')
    ])
    
    return model

# Build model
lstm_model = build_lstm_model((n_timesteps, 1), n_classes)
lstm_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(lstm_model.summary())

In [ ]:
# Train LSTM model
history_lstm = lstm_model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nLSTM Training completed!")

## Model 3: Hybrid CNN-LSTM Model

In [ ]:
def build_hybrid_model(input_shape, n_classes):
    """Build hybrid CNN-LSTM model"""
    model = models.Sequential([
        # CNN layers for feature extraction
        layers.Conv1D(64, kernel_size=7, activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        
        layers.Conv1D(128, kernel_size=5, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Dropout(0.3),
        
        # LSTM layers for temporal dependencies
        layers.LSTM(64, return_sequences=True),
        layers.Dropout(0.3),
        
        layers.LSTM(32),
        layers.Dropout(0.3),
        
        # Dense layers
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(n_classes, activation='softmax')
    ])
    
    return model

# Build model
hybrid_model = build_hybrid_model((n_timesteps, 1), n_classes)
hybrid_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(hybrid_model.summary())

In [ ]:
# Train hybrid model
history_hybrid = hybrid_model.fit(
    X_train, y_train_cat,
    validation_data=(X_val, y_val_cat),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

print("\nHybrid Model Training completed!")

## Training Visualization

In [ ]:
# Plot training history
def plot_training_history(histories, labels):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    colors = ['#E63946', '#4361EE', '#06A77D']
    
    # Accuracy
    for history, label, color in zip(histories, labels, colors):
        axes[0].plot(history.history['accuracy'], label=f'{label} Train', 
                    color=color, linewidth=2)
        axes[0].plot(history.history['val_accuracy'], label=f'{label} Val', 
                    color=color, linewidth=2, linestyle='--')
    
    axes[0].set_title('Model Accuracy', fontsize=14, fontweight='bold')
    axes[0].set_xlabel('Epoch', fontsize=12)
    axes[0].set_ylabel('Accuracy', fontsize=12)
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss
    for history, label, color in zip(histories, labels, colors):
        axes[1].plot(history.history['loss'], label=f'{label} Train', 
                    color=color, linewidth=2)
        axes[1].plot(history.history['val_loss'], label=f'{label} Val', 
                    color=color, linewidth=2, linestyle='--')
    
    axes[1].set_title('Model Loss', fontsize=14, fontweight='bold')
    axes[1].set_xlabel('Epoch', fontsize=12)
    axes[1].set_ylabel('Loss', fontsize=12)
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(
    [history_cnn, history_lstm, history_hybrid],
    ['CNN', 'LSTM', 'Hybrid']
)

## Model Evaluation

In [ ]:
# Evaluate all models
models_dict = {
    'CNN': cnn_model,
    'LSTM': lstm_model,
    'Hybrid': hybrid_model
}

results = {}
for name, model in models_dict.items():
    loss, accuracy = model.evaluate(X_test, y_test_cat, verbose=0)
    y_pred = np.argmax(model.predict(X_test, verbose=0), axis=1)
    
    results[name] = {
        'loss': loss,
        'accuracy': accuracy,
        'predictions': y_pred
    }
    
    print(f"\n{name} Model:")
    print(f"  Test Loss: {loss:.4f}")
    print(f"  Test Accuracy: {accuracy:.4f}")

# Compare models
comparison_df = pd.DataFrame({
    'Model': list(results.keys()),
    'Accuracy': [results[k]['accuracy'] for k in results.keys()],
    'Loss': [results[k]['loss'] for k in results.keys()]
})

print("\nModel Comparison:")
print(comparison_df)

In [ ]:
# Visualize model comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Accuracy comparison
axes[0].bar(comparison_df['Model'], comparison_df['Accuracy'], 
           color=['#E63946', '#4361EE', '#06A77D'], alpha=0.7, edgecolor='black')
axes[0].set_title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Accuracy', fontsize=12)
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3, axis='y')

# Loss comparison
axes[1].bar(comparison_df['Model'], comparison_df['Loss'], 
           color=['#E63946', '#4361EE', '#06A77D'], alpha=0.7, edgecolor='black')
axes[1].set_title('Model Loss Comparison', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Loss', fontsize=12)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated:
- Building and training 1D CNN model for ECG classification
- Building and training LSTM model for temporal pattern recognition
- Building and training hybrid CNN-LSTM model
- Comparing model performance

The trained models are ready for deployment and real-time ECG analysis!